## Using a QMModel: Performing Geometry Optimization

In [1]:
import numpy as np
import ase
from ase.build import molecule
from ase.optimize import LBFGS

In [2]:
from qmlearn.api.api4ase import QMLCalculator
from qmlearn.drivers.mol import QMMol
from qmlearn.io.model import db2qmmodel

##### Get Energies using Delta learning by altering the initial geometry 

1) Open a database that already defined the Linear Regression model using `db2qmmodel`

In [3]:
qmmodel = db2qmmodel('h2o_vib_QML_set.hdf5')

Guess DB names : {'qmmol': 'rks/qmmol', 'atoms': 'rks/train_atoms_36', 'properties': 'rks/train_props_36'}
Guess mmodels: {'gamma': KernelRidge(alpha=0.1), 'd_gamma': LinearRegression(), 'd_energy': LinearRegression(), 'd_forces': LinearRegression()}


2) Define second learning features `energy` and `forces` 

3) Define the Calculator, based on `second_learn` models. 

In [4]:
second_learn = {
        'energy' : 'd_energy',
        'forces' : 'd_forces',
        }
calc = QMLCalculator(qmmodel = qmmodel, second_learn = second_learn, method = 'gamma')

4) Alter the atom_test positions by 0.2 and perform an optimization using `LBFGS` method.

In [5]:
atoms_test=molecule('H2O')
atoms_test.positions[0,0] += 0.2

In [6]:
atoms = atoms_test.copy()
atoms.calc = calc

In [7]:
LBFGS(atoms, trajectory = 'opt.traj').run(fmax=0.01)
atoms.positions

       Step     Time          Energy         fmax
LBFGS:    0 16:28:09    -2070.589811        1.2693
LBFGS:    1 16:28:09    -2070.613697        0.5414
LBFGS:    2 16:28:09    -2070.613824        0.5288
LBFGS:    3 16:28:09    -2070.618347        0.0468
LBFGS:    4 16:28:09    -2070.618368        0.0335
LBFGS:    5 16:28:10    -2070.618410        0.0144
LBFGS:    6 16:28:10    -2070.618419        0.0042


array([[ 1.99138374e-01, -2.48457375e-05,  9.49096628e-02],
       [ 4.31960289e-04,  7.65801951e-01, -4.64831678e-01],
       [ 4.29652834e-04, -7.65777111e-01, -4.64910030e-01]])

##### Get Energies using PySCF engine

1) Define the basis set, level of theory, exchange correlation functional and the total charge of your system

In [8]:
basis = 'cc-pvTZ'
xc = 'lda,vwn_rpa'
method = 'rks'
charge = 0

2) Create atoms `atoms` copy, and the `refqmmol` object to initialize the engine.

3) Initialize the calculator using `QMLCalculator` class.

In [9]:
atoms = atoms_test.copy()
refqmmol = QMMol(atoms = atoms, method = method, basis=basis, xc = xc, charge=charge)
atoms.calc = QMLCalculator(qmmodel = refqmmol, method = 'engine')

4) Run the geometry optimization

In [10]:
LBFGS(atoms, trajectory = 'opt.traj').run(fmax=0.01)
atoms.positions

       Step     Time          Energy         fmax
LBFGS:    0 16:40:06    -2070.596525        1.3259
LBFGS:    1 16:40:09    -2070.616301        0.2563
LBFGS:    2 16:40:13    -2070.617393        0.1126
LBFGS:    3 16:40:17    -2070.618015        0.1030
LBFGS:    4 16:40:21    -2070.618396        0.0495
LBFGS:    5 16:40:25    -2070.618425        0.0106
LBFGS:    6 16:40:28    -2070.618427        0.0005


array([[ 1.92577545e-01,  1.60969174e-15,  9.72371798e-02],
       [ 3.71122733e-03,  7.65804799e-01, -4.66034590e-01],
       [ 3.71122733e-03, -7.65804799e-01, -4.66034590e-01]])

##### Get Energies using Delta learning  without altering the geometry

1) Open a model in a database and keep it in `qmmodel`.

2) Create atoms `atoms` copy.

3) Initialize the calculator using `QMLCalculator` class, based on the define `second_learn` models.

In [11]:
qmmodel = db2qmmodel('./h2o_md_300_QML_set.hdf5')
atoms = atoms_test.copy()
atoms.calc = QMLCalculator(qmmodel = qmmodel, second_learn = second_learn, method = 'gamma')

Guess DB names : {'qmmol': 'rks/qmmol', 'atoms': 'rks/train_atoms_30', 'properties': 'rks/train_props_30'}
Guess mmodels: {'gamma': KernelRidge(alpha=0.1), 'd_gamma': LinearRegression(), 'd_energy': LinearRegression(), 'd_forces': LinearRegression()}


4) Run the geometry optimization

In [12]:
LBFGS(atoms, trajectory = 'opt.traj').run(fmax=0.01)
atoms.positions

       Step     Time          Energy         fmax
LBFGS:    0 16:41:26    -2070.596523        1.3260
LBFGS:    1 16:41:26    -2070.616322        0.2564
LBFGS:    2 16:41:26    -2070.617412        0.1126
LBFGS:    3 16:41:26    -2070.618032        0.1029
LBFGS:    4 16:41:26    -2070.618411        0.0495
LBFGS:    5 16:41:27    -2070.618441        0.0106
LBFGS:    6 16:41:27    -2070.618441        0.0014


array([[ 1.92610265e-01,  1.14244173e-05,  9.72291730e-02],
       [ 3.69633877e-03,  7.65794139e-01, -4.66026200e-01],
       [ 3.69338994e-03, -7.65805574e-01, -4.66034992e-01]])